# Application

News extractive summarizer application

## Loading model to apply to new text

In [ ]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#change current working directory
%cd /content/drive/Shareddrives/Aplicacion inteligencia de la web/Lab2

/content/drive/Shareddrives/Aplicacion inteligencia de la web/Lab2


In [ ]:
## CARGAMOS EL MODELO
!pip install transformers
!pip install seqeval
import json
import torch
import numpy as np
from torch import cuda
from flask import Flask,request,jsonify,render_template
import os
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
import nltk
from nltk import word_tokenize
nltk.download('punkt')

device = 'cuda' if cuda.is_available() else 'cpu'

path = './model/aviation-accidents.pt'
loaded_model = torch.load(path, map_location=torch.device(device))
loaded_model.to(device)

# Save tokenizer?
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL_NAME)

# Load data
f = open('./model/data.json',)
data = json.load(f)
data['index2tag'] = {int(k): v for k,v in data['index2tag'].items()}

f.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Test with 10 own collected data ##

---



In [ ]:
# 10 Example sentences.
sentence1 = 'On July 17, 2014, Malaysia Airlines Flight 17, a Boeing 777-200ER, flying from Amsterdam to Kuala Lumpur, was shot down in an area of Eastern Ukraine near the Ukraine/Russian border during the war in Donbas.'
sentence2 = 'On July 3, 1988, Iran Air Flight 655, an Iranian Airbus A300-200 airliner, was shot down by two surface-to-air missiles from the U.S. Navy guided missile cruiser USS Vincennes over the Strait of Hormuz.'
sentence3 = 'On February 19, 2003, an Iranian military Ilyushin Il-76 crashed in mountainous terrain near Kerman in Iran. The official report says bad weather brought the aircraft down.'
sentence4 = 'On September 1, 1983, a Soviet interceptor Sukhoi Su-15 shot down Korean Air Lines Flight 007, a Boeing 747-230B, bound for Gimpo International Airport in Seoul, South Korea, after it flew into Soviet airspace.'
sentence5 = 'On July 11, 1991, Nigeria Airways Flight 2120, a Douglas DC-8-61 aircraft operated by Nationair Canada, crashed in Jeddah, Saudi Arabia, after two tires ignited upon takeoff, leading to an in-flight fire.'
sentence6 = 'On January 1, 1978, a Boeing 747-237B, Air India Flight 855, crashed into the Arabian Sea just off the coast of Bombay, India, killing all 190 passengers and 23 crew on board.'
sentence7 = 'On April 11, 2018, an Algerian Air Force transport plane crashed shortly after take-off from Boufarik Airport, killing all 247 passengers and 10 crew on board the Ilyushin Il-76.'
sentence8 = 'On September 26, 1997, an Airbus A300B4-220, Garuda Indonesia Flight 152, which departed from Jakarta, Indonesia, and was preparing to land at Medan, North Sumatra, crashed into mountainous terrain, killing 222 passengers and 12 crew members.'
sentence9 = 'On August 6, 1997, a Boeing 747-3B5, Korean Air Flight 801, crashed on approach to the international airport in the United States territory of Guam, killing 228 of the 254 people aboard.'
sentence10 = '1939, August 13 – A Pan Am Sikorsky S-43 crashes into Guanabara Bay, Brazil, due to loss of control following engine failure, killing 12 of the 14 on board.'

sentence_list = [sentence1, sentence2, sentence3, sentence4, sentence5, sentence6, sentence7, sentence8, sentence9, sentence10]

In [ ]:
# Generate output list
context_list = {}
context_list["Year"] = "THE ACCIDENT HAPPENED IN {}"
context_list["DateOfAccident"] = "THE DATE OF THE AVIATION ACCIDENT WAS: {}"
context_list["NumberOfVictims"] = "THERE WERE {} VICTIMS"
context_list["Place"] = "THE ACCIDENT TOOK PLACE IN: {}"
context_list["TypeOfAccident"] = "TYPE OF ACCIDENT WAS: {}"
context_list["TypeOfAircraft"] = "TYPE OF AIRCRAFT WAS: {}"
context_list["Airline"] = "THE AIRLINE WAS: {}"
context_list["FlightNumber"] = "THE FLIGHT NUMBER WAS: {}"

## Application ##


In [ ]:
def render_page(word, tag):
  if tag in context_list:
    print(context_list[tag].format(word))

def predict(sentence):
  inputs = tokenizer(word_tokenize(sentence),
                    is_split_into_words=True, 
                    return_offsets_mapping=True, 
                    padding='max_length', 
                    truncation=True, 
                    max_length=data['max_seq_len'],
                    return_tensors="pt")
  # move to gpu
  ids = inputs["input_ids"].to(device)
  mask = inputs["attention_mask"].to(device)

  outputs = loaded_model(ids, attention_mask=mask)
  logits = outputs[0]

  active_logits = logits.view(-1, loaded_model.num_labels) # shape (batch_size * seq_len, num_labels)
  flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

  tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
  token_predictions = [data['index2tag'][i] for i in flattened_predictions.cpu().numpy()]
  wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

  prediction = []
  for token_pred, mapping in zip(wp_preds, inputs["offset_mapping"].squeeze().tolist()):
    #only predictions on first word pieces are important
    if mapping[0] == 0 and mapping[1] != 0:
      prediction.append(token_pred[1])
    else:
      continue

  # Aplicacion.
  tag_temp = ''
  accumulate_word = ''
  bio = False
  for word, tag in zip(word_tokenize(sentence), prediction):
    # Caso Outside 
    if tag[0] == 'O' and bio:
      render_page(accumulate_word, tag_temp)
      tag_temp = ''
      accumulate_word = ''
      bio = False

    # Caso de dos tag currente != tag anterior.
    elif tag[2:] != tag_temp and bio:
      render_page(accumulate_word, tag_temp)
      # Caso de tag currente es tipo Beganning.
      if tag[0] == 'B':
        tag_temp = tag[2:]
        accumulate_word = word
        bio = True
      else:
        bio = False

    # Caso generico de inside
    elif tag[0] == 'I' and bio:
      accumulate_word = "{} {}".format(accumulate_word, word)
    
    # Caso generico de Beginning
    elif tag[0] == 'B':
      # Caso: B detras de otro B
      if bio:
        render_page(accumulate_word, tag_temp)

      tag_temp = tag[2:]
      accumulate_word = word
      bio = True

    


In [ ]:
# Example Sentence: On January 1, 1978, a Boeing 747-237B, Air India Flight 855, crashed into the Arabian Sea just off the coast of Bombay, India, killing all 190 passengers and 23 crew on board

while True:
  print("_"*100)
  sentence = input("\nREADY FOR YOUR TEXT (QUIT to exit): ")
  if(sentence.lower() == 'quit'):
    print("GOODBYE!")
    print("_"*100)
    break

  print("CALLING THE EXTRACTION SYSTEM...\n")
  predict(sentence)


____________________________________________________________________________________________________

READY FOR YOUR TEXT (QUIT to exit): On July 17, 2014, Malaysia Airlines Flight 17, a Boeing 777-200ER, flying from Amsterdam to Kuala Lumpur, was shot down in an area of Eastern Ukraine near the Ukraine/Russian border during the war in Donbas.
CALLING THE EXTRACTION SYSTEM...

THE DATE OF THE AVIATION ACCIDENT WAS: July 17
THE ACCIDENT HAPPENED IN 2014
THE AIRLINE WAS: Malaysia Airlines
THE FLIGHT NUMBER WAS: Flight 17
TYPE OF AIRCRAFT WAS: Boeing 777-200ER
THE ACCIDENT TOOK PLACE IN: Amsterdam
TYPE OF ACCIDENT WAS: shot down
THE ACCIDENT TOOK PLACE IN: Eastern Ukraine
THE ACCIDENT TOOK PLACE IN: Ukraine/Russian border
THE ACCIDENT TOOK PLACE IN: Donbas
____________________________________________________________________________________________________

READY FOR YOUR TEXT (QUIT to exit): On July 3, 1988, Iran Air Flight 655, an Iranian Airbus A300-200 airliner, was shot down by two su